In [118]:
import numpy as np
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

In [2]:
from datasets import load_dataset
dataset = load_dataset("go_emotions", "simplified")
texts=dataset['train']['text']
labels=dataset['train']['labels']

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(labels)

max_length = 128 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

labels_tensor = tf.constant(encoded_labels, dtype=tf.float32)

num_labels = len(mlb.classes_) 
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metric = tf.keras.metrics.BinaryAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

history = model.fit(
    {'input_ids': tokenized['input_ids'], 'token_type_ids': tokenized['token_type_ids'], 'attention_mask': tokenized['attention_mask']},
    labels_tensor,
    epochs=3,
    batch_size=16,
    validation_split=0.2
)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
2171/2171 [==============================] - 701s 299ms/step - loss: 0.1252 - accuracy: 0.9627 - val_loss: 0.0946 - val_accuracy: 0.9665
Epoch 2/3
2171/2171 [==============================] - 606s 279ms/step - loss: 0.0843 - accuracy: 0.9695 - val_loss: 0.0905 - val_accuracy: 0.9680
Epoch 3/3
2171/2171 [==============================] - 637s 294ms/step - loss: 0.0684 - accuracy: 0.9749 - val_loss: 0.0894 - val_accuracy: 0.9691


In [14]:
test_texts=dataset['train']['text']
test_labels=dataset['train']['labels']
test_encoded_labels = mlb.transform(test_labels)
test_tokenized = tokenizer(test_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
test_labels_tensor = tf.constant(test_encoded_labels, dtype=tf.float32)

evaluation = model.evaluate(
    {'input_ids': test_tokenized['input_ids'], 'token_type_ids': test_tokenized['token_type_ids'], 'attention_mask': test_tokenized['attention_mask']},
    test_labels_tensor
)

1357/1357 [==============================] - 228s 168ms/step - loss: 0.0581 - accuracy: 0.9791


In [29]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

In [125]:
sample_text = "i'm not happy"
sample_tokenized = tokenizer(sample_text, padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

sample_input = {
    'input_ids': sample_tokenized['input_ids'],
    'token_type_ids': sample_tokenized['token_type_ids'],
    'attention_mask': sample_tokenized['attention_mask']
}
predictions=model.predict([sample_input])

1/1 [==============================] - 0s 45ms/step


In [126]:
prd=predictions[0].tolist()
indices = [index for index, value in enumerate(prd[0]) if value > 0]
for i in indices:
    print(LABELS[i])

sadness
